In [1]:
! pip install langchain tiktoken

In [2]:
data_dir = 'data/state_of_the_union/'

In [3]:
import os

paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
texts = [open(path, 'r').read().replace('\n\n', '\n') for path in paths if path.endswith('.txt')]

In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [5]:
text_splitter = CharacterTextSplitter(separator = ".", chunk_size=3000, chunk_overlap=100)
docs = text_splitter.create_documents([texts[0]])

In [6]:
from langchain.prompts.base import BasePromptTemplate
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.text_splitter import TextSplitter
from typing import Any, Dict, Optional, List
from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.summarize import load_summarize_chain, refine_prompts
from langchain.schema import LLMResult, Generation


class RefineLLMChain(LLMChain):
    """Chain to run queries against LLMs. Queries can be large and will be broken up and
    processed incrementally using the Refine summarize strategy. Only one input key allowed.

    Example:
        .. code-block:: python

            from langchain import LLMChain, OpenAI, PromptTemplate
            prompt_template = "Tell me a {adjective} joke"
            prompt = PromptTemplate(
                input_variables=["adjective"], template=prompt_template
            )
            llm = LLMChain(llm=OpenAI(), prompt=prompt)
    """
    
    text_splitter: TextSplitter
    """TextSplitter used to split up the input text if necessary (e.g. input text is too long to use for a single call
    to LLM)
    """
    question_prompt: BasePromptTemplate = refine_prompts.PROMPT
    """Question prompt for refine chain
    """
    refine_prompt: BasePromptTemplate = refine_prompts.REFINE_PROMPT
    """Refine prompt for refine chain
    """

    # def _call(
    #     self,
    #     inputs: Dict[str, Any],
    #     run_manager: Optional[CallbackManagerForChainRun] = None,
    # ) -> Dict[str, str]:
    #     text = inputs[self.prompt.input_variables[0]] # do some input validation here. checking that there is only one key in the dict
    #     docs = self.text_splitter.create_documents([text])
    #     with open("doc_nums.txt", "a") as myfile:
    #         myfile.write(f"appended text {len(docs)}")
    #     refine_chain = load_summarize_chain(self.llm, chain_type="refine")
    #     output = refine_chain({"input_documents": docs}, return_only_outputs=True)
    #     return {'text': output['output_text']}


    def generate( # this is a hack and should be refactored
        self,
        input_list: List[Dict[str, Any]],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> LLMResult:
        """Generate LLM result from inputs."""
        # prompts, stop = self.prep_prompts(input_list, run_manager=run_manager)
        # output = self.llm.generate_prompt(
        #     prompts,
        #     stop,
        #     callbacks=run_manager.get_child() if run_manager else None,
        #     **self.llm_kwargs,
        # )
        refine_chain = load_summarize_chain(
            self.llm, chain_type="refine",
            refine_prompt=self.refine_prompt,
            question_prompt=self.question_prompt, 
        )
        generations = []
        for index, input_item in enumerate(input_list):
            print("processing", index)
            text = input_item[self.prompt.input_variables[0]] # do some input validation here. checking that there is only one key in the dict
            docs = self.text_splitter.create_documents([text])
            output = refine_chain(
                {"input_documents": docs}, return_only_outputs=True
            )
            generations.append([Generation(text=output["output_text"])])
        return LLMResult(generations=generations)
    
    """
    todo: implement async call
    """

In [7]:
llm1 = OpenAI()
llm2 = OpenAI() # can probably just use one variable here

In [8]:
refine_text_splitter = CharacterTextSplitter(separator = ".", chunk_size=1000, chunk_overlap=100)

In [9]:
prompt_template = "{text}"
prompt = PromptTemplate(
    input_variables=["text"], template=prompt_template
) # kinda hacky but works
refineLLMChain = RefineLLMChain(llm=llm1, prompt=prompt, text_splitter=refine_text_splitter)

Test the output of the custom RefineLLMChain

In [10]:
refineLLMChain({"text": docs[0].page_content})

processing 0


{'text': "\n\nIn just three years, America has made a great comeback with a booming economy, soaring incomes, plummeting poverty, falling crime, and increased confidence. The country is also gaining respect on the world stage and is no longer used, taken advantage of, or scorned by other nations. Through reversing the failed economic policies of the previous administration, enacting historic and record-setting tax cuts, and fighting for fair and reciprocal trade agreements, 7 million new jobs have been created and the unemployment rate has reached the lowest levels in history for African Americans, Hispanic Americans, and Asian Americans. Our military is completely rebuilt, with its power being unmatched anywhere in the world, our borders are secure, our families are flourishing, our values are renewed, and our pride is restored. We have the opportunity to build the world's most prosperous and inclusive society, where every citizen can join in America's success and every community can 

In [11]:
refine_chain = load_summarize_chain(llm2, chain_type="refine")

In [12]:
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=refineLLMChain,
    combine_document_chain=refine_chain,
    return_intermediate_steps=True
)

Test the custom MapReduceDocumentChain that uses a refine summarizing strategy for both the map and reduce steps

In [ ]:
output = map_reduce_chain({"input_documents": docs[:3]})

processing 0
processing 1
processing 2


In [ ]:
output

### Now we will use a custom MapReduceDocumentsChain to summarize and contrast the State of the Union speeches of President Obama, Trump, and Biden

In [ ]:
prompt_template = """Write a concise summary of the following:

{text}

CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary constrasting the state of the union speeches of President Biden, Trump, and Obama.\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary."
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

refine_chain = load_summarize_chain(llm2, chain_type="refine", question_prompt=PROMPT, refine_prompt=refine_prompt)

In [ ]:
refine_text_splitter = CharacterTextSplitter(separator = ".", chunk_size=3000, chunk_overlap=100)

In [ ]:
prompt_template = "{text}"
prompt = PromptTemplate(
    input_variables=["text"], template=prompt_template
) # kinda hacky but works
refineLLMChain = RefineLLMChain(llm=llm1, prompt=prompt, text_splitter=refine_text_splitter)

In [ ]:
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=refineLLMChain,
    combine_document_chain=refine_chain,
    return_intermediate_steps=True
)

In [ ]:
president_names = [ path.split('/')[-1].replace('.txt', '') for path in paths if path.endswith('.txt')]

In [ ]:
from langchain.schema import Document

docs = [Document(page_content=f"President {president_names[index]}:\n{text}") for index, text in enumerate(texts)]

In [ ]:
output = map_reduce_chain({"input_documents": docs})

In [ ]:
output['intermediate_steps']

In [ ]:
output['output_text']

Looks like the output was cut short for some reason. Lets try using a StuffChain for the combine chain of the MapReduceChain. The MapReduceChain is recursive if necessary, breaking a longer list of documents (after the initial map step) up into multiple shorter lists that can be processed without LLM context window issues.

In [ ]:
stuff_chain = load_summarize_chain(llm2, chain_type="stuff")

In [ ]:
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=refineLLMChain,
    combine_document_chain=stuff_chain,
    return_intermediate_steps=True
)

In [ ]:
output = map_reduce_chain({"input_documents": docs})